### Ajustaremos los parametros en Vertex AI haciendo uso de hyperparameter_tuning

In [21]:
!python --version

Python 3.9.2


#### El propósito es la implementación de hyperparameter_tuning en la nube
Para definir el modelo por subclassing, el cual recibirá los hiperparámetros
para encontrar los mejores.

In [14]:
!mkdir -p hyper

# Creamos el modelo dentro de una función que recibirá todos los hiperparámetros para la búsqueda.


In [15]:
%%writefile hyper/model.py

import numpy as np
import tensorflow as tf
import os

def hyperparameter_tun(d1,d2,lr,epochs,batch):
    """
    Esta función crea y entrena un modelo utilizando los hiperparámetros proporcionados.

    :param d1: Número de neuronas en la primera capa densa.
    :type d1: int
    :param d2: Número de neuronas en la segunda capa densa.
    :type d2: int
    :param lr: Tasa de aprendizaje para el optimizador Adam.
    :type lr: float
    :param epochs: Número de épocas para entrenar el modelo.
    :type epochs: int
    :param batch: Tamaño del lote para el entrenamiento.
    :type batch: int
    :return: La precisión de validación del modelo entrenado en la última época.
    :rtype: float
    """
    # Subclassing
    class MyModel(tf.keras.Model):  # Hereda de tf.keras.Model.
        def __init__(self, d2):
            super(MyModel, self).__init__()
            self.dense = tf.keras.layers.Dense(d2)

        def call(self, x):
            return self.dense(x)

    class MyModel2(MyModel):  # Hereda de MyModel.
        def __init__(self, d1=10,d2=10):
            super(MyModel2, self).__init__(d2)
            self.dense2 = tf.keras.layers.Dense(d1)
            self.dense3 = tf.keras.layers.Dense(1)

        def call(self, x):
            x = super().call(x)  # Llama al método call de MyModel.
            x = self.dense2(x)
            return self.dense3(x)
        
    model = MyModel2(d1,d2)

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=optimizer, loss='MSE', metrics=['accuracy'])
    
    x_train = np.random.random((1000, 10))
    y_train = np.random.randint(2, size=(1000,))
    x_val = np.random.random((200, 10))
    y_val = np.random.randint(2, size=(200,))

    history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch, validation_data=(x_val, y_val))

    hp_metric = history.history['val_accuracy'][-1]
    # Evaluar el modelo
    x_test = np.random.random((200, 10))
    y_test = np.random.randint(2, size=(200,))
    evv = model.evaluate(x_test, y_test)
    print("HERE: ",evv)
    model.save('model/m1')
    return hp_metric

Overwriting hyper/model.py


#### Este script es el módulo principal que se ejecuta al iniciar la imagen Docker, facilitando la búsqueda automatizada de los hiperparámetros óptimos para un modelo de machine learning en la plataforma AI de Google Cloud.

In [16]:
%%writefile hyper/hyper.py

import tensorflow as tf

import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt
from model import hyperparameter_tun
import argparse
import hypertune

def get_args():
  '''Parses args. Must include all hyperparameters you want to tune.'''

  parser = argparse.ArgumentParser()
  parser.add_argument(
      '--dense1',
      required=True,
      type=int,
      help='dense1')
  parser.add_argument(
      '--dense2',
      required=True,
      type=int,
      help='dense2')
  parser.add_argument(
      '--lr',
      required=True,
      type=float,
      help='learning rate')
  parser.add_argument(
      '--epochs',
      required=True,
      type=int,
      help='epochs')
  parser.add_argument(
      '--batch',
      required=True,
      type=int,
      help='batch')
  args = parser.parse_args()
  return args

def main():
  args = get_args()
  hp_metric = hyperparameter_tun(d1=args.dense1,d2=args.dense2,lr=args.lr,epochs=args.epochs,batch=args.batch)
    
  hpt = hypertune.HyperTune()

  hpt.report_hyperparameter_tuning_metric(
      hyperparameter_metric_tag='accuracy',
      metric_value=hp_metric,
      global_step=args.epochs)


if __name__ == "__main__":
    main()

Overwriting hyper/hyper.py


In [17]:
%%writefile hyper/.dockerignore

__pycache__
*.pyc
*.pyo
*.egg-info
**/model/m0/
**/model/m1/
Dockerfile

env/
.git
.dockerignore
.gitignore

# Jupyter
.ipynb_checkpoints/

# Python
__pycache__/
*.pyc
*.pyo
*.egg-info
*.egg

# Entornos virtuales
venv/
.env/
env
**/__pycache__
__pycache__
*.virtualenv/

# Generales
.DS_Store
*.log
.idea/

Overwriting hyper/.dockerignore


### Creación de un Dockerfile para la imagen de búsqueda de hiperparámetros.


In [18]:
%%writefile hyper/Dockerfile

#FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-8
FROM gcr.io/deeplearning-platform-release/tf-gpu.2-6.py39

WORKDIR /

# Actualiza pip
RUN pip install --upgrade pip
# Copia el archivo requirements.txt al contenedor
COPY requirements.txt /requirements.txt
# Instala las dependencias
RUN pip install -r /requirements.txt
# Installs hypertune library
RUN pip install cloudml-hypertune

# Copies the trainer code to the Docker image.
COPY . .

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "hyper"]

Overwriting hyper/Dockerfile


In [23]:
%%bash
source env/bin/activate && pip freeze > hyper/requirements.txt


In [ ]:
!docker build -t hyper hyper/

In [25]:
!docker tag hyper us-central1-docker.pkg.dev/projecto2-373519/myimages/hyper:v1.0.0

In [ ]:
!docker push us-central1-docker.pkg.dev/projecto2-373519/myimages/hyper:v1.0.0

# Configuración del Tipo de Máquina y Aceleradores

In [2]:
import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import hyperparameter_tuning as hpt

In [ ]:
!gcloud auth application-default login

In [14]:
aiplatform.init(location="us-central1")

In [19]:
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": "n1-standard-4",
            "accelerator_type": None, #"NVIDIA_TESLA_T4",
            "accelerator_count": None,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": "us-central1-docker.pkg.dev/projecto2-373519/myimages/hyper:v1.0.0"
        },
    }
]

In [23]:
my_custom_job = aiplatform.CustomJob(
    display_name="tunning",
    worker_pool_specs=worker_pool_specs,
    staging_bucket="gs://models_ai_save_2/tunning",
)

# Configurando Hiperparámetros

In [24]:
metric_spec = {"accuracy": "maximize"}
parameter_spec = {
    "dense1": hpt.DiscreteParameterSpec(values=[5, 6], scale=None),
    "dense2": hpt.DiscreteParameterSpec(values=[4, 5, 6], scale=None),
    "lr": hpt.DoubleParameterSpec(min=0.001, max=1, scale="log"),
    "epochs": hpt.DiscreteParameterSpec(values=[2, 3, 4], scale=None),
    "batch": hpt.DiscreteParameterSpec(values=[8, 16, 32], scale=None),
}

In [ ]:
hp_job = aiplatform.HyperparameterTuningJob(
    display_name="tunning_params",
    custom_job=my_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=3,
    parallel_trial_count=3,
)

hp_job.run()

# FIN